<a href="https://colab.research.google.com/github/Yan-Weijia/CompLing/blob/master/HW03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [58]:
import re
from string import punctuation
from collections import Counter, defaultdict
import nltk
nltk.download("punkt")
from nltk import sent_tokenize
import gzip
import csv
punctuation += "«»—…“”"
punct = set(punctuation)
import itertools
!pip install pymorphy2[fast]
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [59]:
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
!wget https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
!wget https://github.com/yutkin/Lenta.Ru-News-Dataset/releases/download/v1.0/lenta-ru-news.csv.gz

--2019-11-27 16:54:05--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/sents_with_mistakes.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 123167 (120K) [text/plain]
Saving to: ‘sents_with_mistakes.txt.2’

sents_with_mistakes 100%[===================>] 120.28K  --.-KB/s    in 0.04s   

2019-11-27 16:54:06 (3.16 MB/s) - ‘sents_with_mistakes.txt.2’ saved [123167/123167]

--2019-11-27 16:54:08--  https://raw.githubusercontent.com/mannefedov/compling_nlp_hse_course/master/data/correct_sents.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent,

# Задание 1

In [0]:
bad = open('sents_with_mistakes.txt', encoding='utf8').read().splitlines()
true = open('correct_sents.txt', encoding='utf8').read().splitlines()

In [0]:
def norm_txt(text):
  new_txt = []
  for sent in text:
    tokens = sent.lower().split()
    tokens = [re.sub('(^\W+|\W+$)', '', tkn) for tkn in tokens if (set(tkn)-punct)]
    new_txt.append(tokens)
  return new_txt

In [0]:
badlist = norm_txt(bad)
truelist = norm_txt(true)

In [63]:
print(truelist[:5])
print(badlist[:5])

[['симпатичнейшее', 'шпионское', 'устройство', 'такой', 'себе', 'гламурный', 'фотоаппарат', 'девушки', 'бонда', 'миниатюрная', 'модель', 'камеры', 'superheadz', 'clap', 'camera'], ['апофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях'], ['поясним', 'эту', 'мысль'], ['получатся', 'вот', 'такие', 'язычки'], ['в', 'массе', 'своей', 'они', 'конечно', 'все', 'очень', 'милые']]
[['симпатичнейшое', 'шпионское', 'устройство', 'такой', 'себе', 'гламурный', 'фотоаппарат', 'девушки', 'бонда', 'миниатюрная', 'модель', 'камеры', 'superheadz', 'clap', 'camera'], ['опофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях'], ['пояним', 'эту', 'мысль'], ['полчатся', 'вот', 'такие', 'язычки'], ['в', 'массе', 'своей', 'они', 'конечно', 'все', 'оччччень', 'милые']]


In [0]:
f = open('corpus_5000.txt', 'w')
with gzip.open('lenta-ru-news.csv.gz', 'rt') as archive:
    reader = csv.reader(archive, delimiter=',', quotechar='"')
    for i, line in enumerate(reader):
        if i < 5000:
            f.write(line[2].replace('\xa0', ' ') + '\n')

In [0]:
def normalize(text):
    normalized_text = [(word.strip(punctuation)) for word in text.lower().split()] 
    normalized_text = [word for word in normalized_text if word] 
    return normalized_text

In [0]:
corpus = []
for text in open('corpus_5000.txt').read().splitlines():
    sents = sent_tokenize(text)
    norm_sents = [normalize(sent) for sent in sents]
    corpus += norm_sents

In [0]:
WORDS = Counter() 
for sent in corpus:
    WORDS.update(sent)
dict_corp = dict(WORDS) 

In [68]:
WORDS.most_common(10)

[('в', 41930),
 ('и', 20346),
 ('на', 17455),
 ('что', 11631),
 ('с', 9616),
 ('по', 8778),
 ('не', 7696),
 ('из', 4369),
 ('он', 4322),
 ('о', 3903)]

In [0]:
vocab = set() 

for sent in corpus:
    vocab.update(sent)

In [0]:
def deletes(word):
    splits     = [(word[:i], word[i:])    for i in range(len(word) + 1)]
    deletes    = [L + R[1:]               for L, R in splits if R] 
    return list(set(deletes))

In [0]:
dict_corp2 = {}
for word in dict_corp:
  for delword in deletes(word):
    if delword not in dict_corp2:
      dict_corp2[delword] = [word]
    else:
      dict_corp2[delword].append(word)

In [0]:
def correction(badword):
  if badword in dict_corp:
    corrected = badword
  else:
    bad_forms = deletes(badword)
    variants = set()
    for form in bad_forms:
      if form in dict_corp2:
        variants.update(dict_corp2[form])
    dict_vars = {} 
    if variants:
      dict_vars = {var:dict_corp[var] for var in list(variants)}
      corrected = max(dict_vars, key=dict_vars.get)
    else:
      corrected = badword
  return corrected

In [0]:
corrected_list = []
for sent in badlist:
  for word in sent:
    corrected_list.append(correction(word))

In [74]:
print(corrected_list[:20])

['симпатичнейшое', 'шпионской', 'устройство', 'такой', 'себе', 'гламурный', 'фотоаппарат', 'девушки', 'бонда', 'миниатюрная', 'модель', 'камеры', 'superheadz', 'camp', 'camera', 'апофеозом', 'дня', 'для', 'меня', 'сегодня']


In [75]:
%%time
print(correction('сонце'))

конце
CPU times: user 186 µs, sys: 971 µs, total: 1.16 ms
Wall time: 1.06 ms


In [76]:
%%time
correction('солнце')

CPU times: user 7 µs, sys: 0 ns, total: 7 µs
Wall time: 10.3 µs


'солнце'

In [77]:
%%time
correction('насмехатьсяаававттававаываываы')

CPU times: user 52 µs, sys: 2 µs, total: 54 µs
Wall time: 55.8 µs


'насмехатьсяаававттававаываываы'

In [0]:
def align_words(sent_1, sent_2):
    tokens_1 = sent_1.lower().split()
    tokens_2 = sent_2.lower().split()
    
    tokens_1 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_1 if (set(token)-punct)]
    tokens_2 = [re.sub('(^\W+|\W+$)', '', token) for token in tokens_2 if (set(token)-punct)]
    
    return list(zip(tokens_1, tokens_2))

In [79]:
print(align_words(true[1], bad[1]))

[('апофеозом', 'опофеозом'), ('дня', 'дня'), ('для', 'для'), ('меня', 'меня'), ('сегодня', 'сегодня'), ('стала', 'стала'), ('фраза', 'фраза'), ('услышанная', 'услышанная'), ('в', 'в'), ('новостях', 'новостях')]


In [80]:
correct = 0 
total = 0 

total_mistaken = 0 
mistaken_fixed = 0 

total_correct = 0 
correct_broken = 0 

cashed = {}
for i in range(len(true)):
    word_pairs = align_words(true[i], bad[i]) 
    for pair in word_pairs: 
        total += 1 
        predicted = cashed.get(pair[1], correction(pair[1]))
        cashed[pair[0]] = predicted 
        if predicted == pair[0]: 
            correct += 1 
        
        if pair[0] == pair[1]: 
            total_correct += 1 
            if pair[0] !=  predicted: 
                correct_broken += 1 
        else: 
            total_mistaken += 1 
            if pair[0] == predicted:
                mistaken_fixed += 1 
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [81]:
print(correct/total) # 订正后总对的数/总词数
print(mistaken_fixed/total_mistaken) # 改对了数/总错词数
print(correct_broken/total_correct) # 改错了数/本来是对的数

0.6821178821178822
0.1987720644666155
0.24554955782703572


In [0]:
mistakes = [] 
total = 0 
for i in range(len(true)): 
    word_pairs = align_words(true[i], bad[i]) 
    for pair in word_pairs:
        total += 1 
        if pair[0] != pair[1]: 
            mistakes.append(pair)

In [83]:
[(wt[0], wt[1], correction(wt[1])) for wt, _ in Counter(mistakes).most_common(10)] 

[('сегодня', 'седня', 'седан'),
 ('вообще', 'вобще', 'общей'),
 ('вообще', 'ваще', 'чаще'),
 ('естественно', 'естесственно', 'естесственно'),
 ('хочется', 'хочеться', 'хочеться'),
 ('кстати', 'кстате', 'статье'),
 ('очень', 'ооочень', 'ооочень'),
 ('как-то', 'както', 'актом'),
 ('очень', 'оооочень', 'оооочень'),
 ('это', 'ето', 'что')]

# Задание 2

In [0]:
def add_tags(text):
  tag_sents = [['<start>', '<start>'] + sent + ['<end>'] for sent in text]
  return tag_sents

In [0]:
sentences_corpus = add_tags(corpus)
sentences_badlist = add_tags(badlist) # [[s,s,词，词，词，词,e]，[s,词，词，词,e]]

In [86]:
print(sentences_corpus[:5])
print(sentences_badlist[:5])

[['<start>', '<start>', 'text', '<end>'], ['<start>', '<start>', 'вице-премьер', 'по', 'социальным', 'вопросам', 'татьяна', 'голикова', 'рассказала', 'в', 'каких', 'регионах', 'россии', 'зафиксирована', 'наиболее', 'высокая', 'смертность', 'от', 'рака', 'сообщает', 'риа', 'новости', '<end>'], ['<start>', '<start>', 'по', 'словам', 'голиковой', 'чаще', 'всего', 'онкологические', 'заболевания', 'становились', 'причиной', 'смерти', 'в', 'псковской', 'тверской', 'тульской', 'и', 'орловской', 'областях', 'а', 'также', 'в', 'севастополе', '<end>'], ['<start>', '<start>', 'вице-премьер', 'напомнила', 'что', 'главные', 'факторы', 'смертности', 'в', 'россии', 'рак', 'и', 'болезни', 'системы', 'кровообращения', '<end>'], ['<start>', '<start>', 'в', 'начале', 'года', 'стало', 'известно', 'что', 'смертность', 'от', 'онкологических', 'заболеваний', 'среди', 'россиян', 'снизилась', 'впервые', 'за', 'три', 'года', '<end>']]
[['<start>', '<start>', 'симпатичнейшое', 'шпионское', 'устройство', 'такой',

In [0]:
def ngrammer(tokens, n):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(tuple(tokens[i:i+n]))
    return ngrams

In [0]:
bigrams = Counter()
trigrams = Counter()

for sentence in sentences_corpus:
  bigrams.update(ngrammer(sentence, 2))
  trigrams.update(ngrammer(sentence, 3))

In [0]:
trigrams_badlist = [ngrammer(sentence, 3) for sentence in sentences_badlist]

In [98]:
print(list(trigrams.items())[:5])
print(list(bigrams.items())[:5])

[(('<start>', '<start>', 'text'), 1), (('<start>', 'text', '<end>'), 1), (('<start>', '<start>', 'вице-премьер'), 12), (('<start>', 'вице-премьер', 'по'), 2), (('вице-премьер', 'по', 'социальным'), 2)]
[(('<start>', '<start>'), 59152), (('<start>', 'text'), 1), (('text', '<end>'), 1), (('<start>', 'вице-премьер'), 12), (('вице-премьер', 'по'), 6)]


In [99]:
print(trigrams_badlist[:5])

[[('<start>', '<start>', 'симпатичнейшое'), ('<start>', 'симпатичнейшое', 'шпионское'), ('симпатичнейшое', 'шпионское', 'устройство'), ('шпионское', 'устройство', 'такой'), ('устройство', 'такой', 'себе'), ('такой', 'себе', 'гламурный'), ('себе', 'гламурный', 'фотоаппарат'), ('гламурный', 'фотоаппарат', 'девушки'), ('фотоаппарат', 'девушки', 'бонда'), ('девушки', 'бонда', 'миниатюрная'), ('бонда', 'миниатюрная', 'модель'), ('миниатюрная', 'модель', 'камеры'), ('модель', 'камеры', 'superheadz'), ('камеры', 'superheadz', 'clap'), ('superheadz', 'clap', 'camera'), ('clap', 'camera', '<end>')], [('<start>', '<start>', 'опофеозом'), ('<start>', 'опофеозом', 'дня'), ('опофеозом', 'дня', 'для'), ('дня', 'для', 'меня'), ('для', 'меня', 'сегодня'), ('меня', 'сегодня', 'стала'), ('сегодня', 'стала', 'фраза'), ('стала', 'фраза', 'услышанная'), ('фраза', 'услышанная', 'в'), ('услышанная', 'в', 'новостях'), ('в', 'новостях', '<end>')], [('<start>', '<start>', 'пояним'), ('<start>', 'пояним', 'эту')

In [0]:
def needed_trigram(ngram, variants):
    dict_vars = {}
    t_grs = [(ngram[0], ngram[1], variant) for variant in list(variants)]
    for tgram in t_grs:
        if tgram[:2] in bigrams:
            freq_tr = trigrams[tgram]/bigrams[tgram[:2]]
            if freq_tr != 0: 
                dict_vars[tgram[-1]] = freq_tr
    
    if not dict_vars:  
        dict_vars = {var:dict_corp[var] for var in list(variants)}
    
    return max(dict_vars, key=dict_vars.get)

In [0]:
def trigrams_correction(ngrams): 
    corrected_sent = []
    for ngram in ngrams: # ('<start>', '<start>', 'симпатичнейшое'),
        if ngram[-1] in dict_corp: 
            corrected_sent.append(ngram[-1])
        else: 
            badword = ngram[-1]
            bad_forms = deletes(badword)
            variants = set()
            for form in bad_forms:
                if form in dict_corp2:
                  variants.update(dict_corp2[form])
            if variants:
              corrected_sent.append(needed_trigram(ngram, variants))
            else:
              corrected_sent.append(badword)
              
    return corrected_sent[:-1]

In [0]:
corrected_sents = []
for sent in trigrams_badlist: 
    corrected_sent = trigrams_correction(sent)
    corrected_sents.append(corrected_sent)

In [100]:
print(corrected_sents[:5])

[['симпатичнейшое', 'шпионской', 'устройство', 'такой', 'себе', 'гламурный', 'фотоаппарат', 'девушки', 'бонда', 'миниатюрная', 'модель', 'камеры', 'superheadz', 'camp', 'camera'], ['апофеозом', 'дня', 'для', 'меня', 'сегодня', 'стала', 'фраза', 'услышанная', 'в', 'новостях'], ['помним', 'эту', 'мысль'], ['полчатся', 'вот', 'такие', 'язычки'], ['в', 'массе', 'своей', 'они', 'конечно', 'все', 'оччччень', 'милые']]


In [96]:
correct = 0 
total = 0 

total_mistaken = 0 
mistaken_fixed = 0 

total_correct = 0 
correct_broken = 0 

cashed = {}
for i in range(len(true)): 
    word_pairs = align_words(true[i], bad[i]) 
    sent_corrected = corrected_sents[i] 
    for j in range(len(word_pairs)): 
        total += 1
        pair = word_pairs[j]
        predicted = cashed.get(pair[1], sent_corrected[j]) 
        cashed[pair[0]] = predicted 
        if predicted == pair[0]: 
            correct += 1 
        
        if pair[0] == pair[1]: 
            total_correct += 1 
            if pair[0] !=  predicted: 
                correct_broken += 1 
        else: 
            total_mistaken += 1
            if pair[0] == predicted: 
                mistaken_fixed += 1 
        
    if not i % 100:
        print(i)
        

0
100
200
300
400
500
600
700
800
900


In [97]:
print(correct/total)
print(mistaken_fixed/total_mistaken)
print(correct_broken/total_correct)

0.685014985014985
0.20721412125863392
0.24348225565636844


# Оценка
Триграммная модель немного улучшила исправление